In [100]:
import pandas as pd
titanic_train = pd.read_csv('train.csv')
titanic_test = pd.read_csv('test.csv')

In [101]:
titanic_train.head()
#乘客序号 是否存活 船票等级(1为头等) 名字 性别 年龄 船上的兄弟姐妹配偶数 船上的父母孩子数 票号 票价 船舱号 登船港口(C,Q,S)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [102]:
titanic_test.head()
#乘客序号 船票等级(1为头等) 名字 性别 年龄 船上的兄弟姐妹配偶数 船上的父母孩子数 票号 票价 船舱号 登船港口(C,Q,S) 

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [103]:
titanic_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [104]:
titanic_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


数值型age,SibSp,Parch,Ticket,Fare;类别型Pclass,Sex,Embarked

除此之外:train中,Age和Embarked有少量缺失;test中,Age和Fare有少量缺失;cabin在二者中均大量缺失

### 数据处理

用平均值填充少量缺失的特征,删除cabin

### 选择特征

船票等级pclass 性别sex 年龄age

In [105]:
train = titanic_train[['Pclass','Sex','Age','Name']].copy()
test = titanic_test[['Pclass','Sex','Age','Name']].copy()
print(test)

     Pclass     Sex   Age                                          Name
0         3    male  34.5                              Kelly, Mr. James
1         3  female  47.0              Wilkes, Mrs. James (Ellen Needs)
2         2    male  62.0                     Myles, Mr. Thomas Francis
3         3    male  27.0                              Wirz, Mr. Albert
4         3  female  22.0  Hirvonen, Mrs. Alexander (Helga E Lindqvist)
..      ...     ...   ...                                           ...
413       3    male   NaN                            Spector, Mr. Woolf
414       1  female  39.0                  Oliva y Ocana, Dona. Fermina
415       3    male  38.5                  Saether, Mr. Simon Sivertsen
416       3    male   NaN                           Ware, Mr. Frederick
417       3    male   NaN                      Peter, Master. Michael J

[418 rows x 4 columns]


In [106]:
train['Title'] = train['Name'].str.extract(r'([A-Za-z]+)\.' , expand = False)#读取姓名中隐含的人物社会地位的信息,比如Mr. Master，
test['Title'] = test['Name'].str.extract(r'([A-Za-z]+)\.' , expand = False)#读取姓名中隐含的人物社会地位的信息,比如Mr. Master，
#标准化称谓
title_mapping = {'Mlls': 'Miss' ,  'Ms':'Miss' , 'Mme':'Mrs'}
train['Title'] = train['Title'].replace(title_mapping)
#处理非主流称谓
title_mask = ~train['Title'].isin(['Mr' , 'Miss' , 'Mrs' , 'Master'])
train.loc[title_mask, 'Title'] = train.loc[title_mask , 'Sex'].map({'male':'Mr' , 'female':'Mrs'})
# 标准化称谓
title_mapping = {'Mlls': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs'}
test['Title'] = test['Title'].replace(title_mapping)

# 处理非主流称谓
title_mask = ~test['Title'].isin(['Mr', 'Miss', 'Mrs', 'Master'])
test.loc[title_mask, 'Title'] = test.loc[title_mask, 'Sex'].map({'male': 'Mr', 'female': 'Mrs'})

train['Age'] = train['Age'].fillna(train['Age'].mean())
test['Age'] = test['Age'].fillna(test['Age'].mean())
title_age_medians = {
    'Mr': 32.32,
    'Miss': 21.68,
    'Mrs': 35.86,
    'Master': 4.57
}
for title, median_age in title_age_medians.items():
    age_mask = (test['Age'].isnull()) & (test['Title'] == title)
    test.loc[age_mask, 'Age'] = median_age
for title, median_age in title_age_medians.items():
    age_mask = (train['Age'].isnull()) & (train['Title'] == title)
    train.loc[age_mask, 'Age'] = median_age

In [107]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Pclass  891 non-null    int64  
 1   Sex     891 non-null    object 
 2   Age     891 non-null    float64
 3   Name    891 non-null    object 
 4   Title   891 non-null    object 
dtypes: float64(1), int64(1), object(3)
memory usage: 34.9+ KB


In [108]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Pclass  418 non-null    int64  
 1   Sex     418 non-null    object 
 2   Age     418 non-null    float64
 3   Name    418 non-null    object 
 4   Title   418 non-null    object 
dtypes: float64(1), int64(1), object(3)
memory usage: 16.5+ KB


In [109]:
train = train.drop(['Name','Sex'], axis=1)
test = test.drop(['Name','Sex'], axis=1)

In [110]:
from sklearn.model_selection import train_test_split
target = titanic_train['Survived'].copy()
# one-hot
features = pd.get_dummies(train , columns=['Pclass', 'Title', ] , drop_first = True)
print(features.head())
Train_features, Test_features, Train_target, Test_target = train_test_split(features , target , test_size = 0.25 , random_state = 33)

    Age  Pclass_2  Pclass_3  Title_Miss  Title_Mr  Title_Mrs
0  22.0     False      True       False      True      False
1  38.0     False     False       False     False       True
2  26.0     False      True        True     False      False
3  35.0     False     False       False     False       True
4  35.0     False      True       False      True      False


### 决策树

In [111]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

dtree = DecisionTreeClassifier(random_state=33)
dtree.fit(Train_features, Train_target)

predictions = dtree.predict(Test_features)
print("准确率 (Accuracy):", accuracy_score(Test_target, predictions))
print("\n分类报告 (Classification Report):\n", classification_report(Test_target, predictions))
print("\n编码后的特征列名：", features.columns.tolist())

准确率 (Accuracy): 0.8385650224215246

分类报告 (Classification Report):
               precision    recall  f1-score   support

           0       0.85      0.89      0.87       134
           1       0.82      0.76      0.79        89

    accuracy                           0.84       223
   macro avg       0.83      0.83      0.83       223
weighted avg       0.84      0.84      0.84       223


编码后的特征列名： ['Age', 'Pclass_2', 'Pclass_3', 'Title_Miss', 'Title_Mr', 'Title_Mrs']


### 提交

In [112]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Pclass  418 non-null    int64  
 1   Age     418 non-null    float64
 2   Title   418 non-null    object 
dtypes: float64(1), int64(1), object(1)
memory usage: 9.9+ KB


In [113]:
test = pd.get_dummies(test , columns=['Pclass', 'Title', ] , drop_first = True)

test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Age         418 non-null    float64
 1   Pclass_2    418 non-null    bool   
 2   Pclass_3    418 non-null    bool   
 3   Title_Miss  418 non-null    bool   
 4   Title_Mr    418 non-null    bool   
 5   Title_Mrs   418 non-null    bool   
dtypes: bool(5), float64(1)
memory usage: 5.4 KB


In [114]:
dtree2 = DecisionTreeClassifier(random_state=33)
dtree2.fit(features, target)

predictions = dtree2.predict(test)
sequence_numbers = range(892 , 892 + len(predictions))
output_df = pd.DataFrame({
    'PassengerId': sequence_numbers,
    'Survived': predictions
})
output_df.to_csv('predictions.csv', index=False)